In [3]:
from __future__ import print_function
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
# number 1 to 10 data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [4]:
#hyperparameter
lr =0.001
training_iter = 10000
batch_size = 128
display_step = 10

n_inputs = 28 #MNIST data input (img shape:28*28)
n_steps = 28 # time step
n_hidden_units = 128 #neurous in hidden layer
n_classes = 10 #MNIST classes (0-9)



In [5]:
# tf graph input
x = tf.placeholder(tf.float32,[None,n_steps,n_inputs])
y = tf.placeholder(tf.float32,[None,n_classes])


In [6]:
# define weight
weights ={
    #(28*128)
    'in':tf.Variable(tf.random_normal([n_inputs, n_hidden_units])),
    #(128*10)
    'out':tf.Variable(tf.random_normal([n_hidden_units, n_classes]))
}

biaes = {
    #(128,)
    'in':tf.Variable(tf.constant(0.1,shape=[n_hidden_units,])),
    #(10,)
    'out':tf.Variable(tf.constant(0.1,shape=[n_classes,]))
}

In [9]:
#RNN
def RNN (X,weights, biaes):
    #hidden layer for input to cell
    #X(128 bacth ,28 step , 28 inputs)
    # ==> (128*28, 28 inputs)
    ########################################################input layer
    X = tf.reshape(X, [-1, n_inputs])
    X_in = tf.matmul(X,weights['in'])+biaes['in']
    #==> (128 batch *28,128 hidden)
    X_in = tf.reshape(X_in,[-1,n_steps,n_hidden_units])
    
    #cell
    ##########################################################cell
    
    lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden_units)
    # lstm cell is divided into two parts(c_state,m_state)
    
    _init_state = lstm_cell.zero_state(batch_size,dtype=tf.float32)
    
    #calculate
    # 每一步的output 都在outputs中， 这是一个list。 state 是最后一个state
    #time major 指的是 n_step的维度是不是放在第一个维度
    #outputs,states = tf.nn.dynamic_rnn(lstm_cell,X_in,initial_state=_init_state, time_major = False)
    outputs, final_state = tf.nn.dynamic_rnn(lstm_cell, X_in, initial_state=_init_state, time_major=False)
    # hidden layer for output as the final result
    ######################################################output layer
    # states[1]==outputs[-1]
   # results = tf.matmul(states[1],weights['out'])+biaes['out']
    outputs = tf.unstack(tf.transpose(outputs, [1,0,2]))
    results = tf.matmul(outputs[-1], weights['out']) + biaes['out'] 
    return results
    
    
    

In [10]:
pred = RNN(x, weights, biaes)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
train_op = tf.train.AdamOptimizer(lr).minimize(cost)



ValueError: Variable rnn/basic_lstm_cell/kernel already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "<ipython-input-7-8bfcde6b86c3>", line 24, in RNN
    outputs, final_state = tf.nn.dynamic_rnn(lstm_cell, X_in, initial_state=_init_state, time_major=False)
  File "<ipython-input-8-ff28abc03d17>", line 1, in <module>
    pred = RNN(x, weights, biaes)
  File "/Users/james/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2910, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
correct_pred = tf.equal(tf.arg_max(pred,1),(tf.arg_max(y,1)))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [11]:
with tf.Session() as sess:
    init = tf.global_variables_initializer
    sess.run(init)
    step=0
    while step*batch_size < training_iters:
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape([batch_size, n_steps, n_inputs])
        sess.run([train_op],feed_dict={xs:batch_xs,ys:batch_ys})
        if step % 50 ==0 :
            print(sess.run(accuracy, feed_dict={x:batch_xs,y:batch_ys}))
        step+=1

TypeError: Fetch argument <function global_variables_initializer at 0x10d8948c8> has invalid type <class 'function'>, must be a string or Tensor. (Can not convert a function into a Tensor or Operation.)